In [1]:
import os
import mysql.connector
from dotenv import load_dotenv
import pandas as pd

In [2]:
# Load environment variables
load_dotenv('./../mysecrets.env')
PASSWORD = os.getenv('PASSWORD')
IP = os.getenv('IP')

# Database connection parameters
username = 'valente'
password = PASSWORD
host = IP
port = '3306'
database_name = 'DV_student_netlixProject'

# Load data into pandas
df_titles = pd.read_csv('./../Data/complete_titles.csv')
df_credits = pd.read_csv('./../Data/raw_credits.csv')

# Establish MySQL connection
connection = mysql.connector.connect(
    host=host,
    user=username,
    password=password,
    port=port
)
cursor = connection.cursor()

# Drop and recreate the database
try:
    cursor.execute(f"DROP DATABASE IF EXISTS {database_name}")
    print(f"Database {database_name} dropped.")
    cursor.execute(f"CREATE DATABASE {database_name}")
    print(f"Database {database_name} created.")
except mysql.connector.Error as err:
    print(f"Error: {err}")

# Use the new database
try:
    cursor.execute(f"USE {database_name}")
    print(f"Using database {database_name}.")
except mysql.connector.Error as err:
    print(f"Error: {err}")

# Close the connection
#cursor.close()
#connection.close()

Database DV_student_netlixProject dropped.
Database DV_student_netlixProject created.
Using database DV_student_netlixProject.


# creating Relational, MySQL Database

In [3]:
table_names = ['Titles', 'Genres','Countries', 'Roles', 'Persons']

def delete_tables(table_names:list) -> None:  
    for table in table_names:
        try:
            cursor.execute(f"DROP TABLE IF EXISTS {table}")
            print(f"Database {table} dropped.")
        except mysql.connector.Error as err:
            print(f"Error: {err}")


In [4]:
# Create Titles table
while True:
    try:
        cursor.execute("""
        CREATE TABLE Titles (
            title_id INT,
            title_name VARCHAR(100) NOT NULL,
            release_year INT NOT NULL,
            media_type_is_movie BOOLEAN,
            show_seasons INT,
            title_runtime INT,
            votes_imdb INT,
            score_imdb FLOAT,
            is_awarded BOOLEAN,
            is_best BOOLEAN,
                                    
            PRIMARY KEY (title_id)
        );
        """)

        # Create Genres Table
        cursor.execute( """
        CREATE TABLE Genres(
            Genre_name VARCHAR(50),
            Title_id INT,
                            
            PRIMARY KEY (Title_id, Genre_name),
            FOREIGN KEY (Title_id) REFERENCES Titles(Title_id)
        ); """)

        # Create Country Table
        cursor.execute( """
        CREATE TABLE Countries(
            Country_code VARCHAR(10),
            Title_id INT,
                            
            PRIMARY KEY (Title_id, Country_code),
            FOREIGN KEY (Title_id) REFERENCES Titles(Title_id)
        ); """)

        # Create Persons Table
        cursor.execute( """ 
        CREATE TABLE Persons(
            person_id INT,
            personName VARCHAR(50),
                            
            PRIMARY KEY (person_id)); """)

        # Crate Roles Table
        cursor.execute( """ 
        CREATE TABLE Roles(
            role_id INT AUTO_INCREMENT,
            person_id INT NOT NULL,
            title_id INT NOT NULL,
            role_character VARCHAR(100),
            role_isActor BOOLEAN,
                            
            PRIMARY KEY (role_id),
            FOREIGN KEY (person_id) REFERENCES Persons(person_id),
            FOREIGN KEY (title_id) REFERENCES Titles(title_id)
        );""")
        break

    except mysql.connector.Error as err:
        print(err)
        delete_tables(table_names)